In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import numpy as np
import holodeck as holo
import holodeck.single_sources as ss
import holodeck.cyutils as cy
import matplotlib.pyplot as plt

### grid:

1x  |  xx  |  xx  |  xx  |  xx \
xx  |  9x  |  x3  |  xx  |  xx \
xx  |  xx  |  xx  |  x4  |  x5 

10x  |  x80   |  xx   |  xx   |  xx \
xx   |  xx  |  x30  |  xx   |  xx \
xx   |  xx   |  xx   |  x40  |  x50 

In [ ]:
hsamp = np.array([[[[[1,10],[0,80],[0,6],[0,6],[0,6]],
                [[0,6],[9,6],[0,6],[0,6],[0,6]],
                [[0,6],[0,6],[0,6],[0,6],[0,6]]]],

                [[[[0,6],[0,6],[0,6],[0,6],[0,6]],
                [[0,6],[0,6],[3,30],[0,6],[0,6]],
                [[0,6],[0,6],[0,6],[4,40],[5,50]]]]])
bgnum = np.full_like(hsamp, 100)

print(hsamp.shape)
print(bgnum.shape)
# print(grid)
hsone=hsamp[...,0]
print(hsone.shape)

In [ ]:
repped = np.repeat(hsone[...,np.newaxis],2,axis=4)
print(repped.shape)

In [ ]:
# one real
hsmax = np.amax(hsone, axis=(0,1,2))
shape = hsone.shape
print(shape)
newshape = (shape[0]*shape[1]*shape[2], shape[3])
print(newshape)
hsone = hsone.reshape(newshape) # change hsamp to shape (M-1*Q-1*Z-1, F-1)
argmax = np.argmax(hsone, axis=0) # max at each frequency & realization
hsone = hsone.reshape(shape) # restore hsamp shape to (M-1, Q-1, Z-1, F-1)
print(hsone.shape)
mqz = np.array(np.unravel_index(argmax, shape[:-1])) # unravel indices
print('mqz', mqz.shape)
print(mqz)
f_ids = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int) # frequency indices
print(np.append(mqz, f_ids))
ssnew = np.append(mqz, f_ids).reshape(4,len(f_ids))
print('ssnew', ssnew)
# print(hsone[ssnew[0], ssnew[1], ssnew[2], ssnew[3]])

In [ ]:
#hsamp for all bins at all realizations

#gets max for each frequency and realization
max = np.amax(hsamp, axis=(0,1,2))
print(max)
shape = hsamp.shape
print('shape',shape)
newshape = (shape[0]*shape[1]*shape[2], shape[3], shape[4])
print('newshape', newshape)
hsamp = hsamp.reshape(newshape)
argmax = np.argmax(hsamp, axis=0)
hsamp = hsamp.reshape(shape)

# shape (3, F, R)
mqz = np.array(np.unravel_index(argmax, shape[:-2])) # unravel indices
print('mqz', mqz.shape)


# I GOt IT!!!!!
# MQZ in shape (3,F,R)
# Want to get MQZF in shape (4,F,R)
# Need f_ids in shape (1,F,R) to append

f_ids = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int) # frequency indices
f_ids = np.repeat(f_ids, R).reshape(F,R)[np.newaxis,:]
ssnew = np.append(mqz,f_ids, axis=0)
print(ssnew, ssnew.shape)

In [ ]:
# print(hsamp[ssnew[0,:,0], ssnew[1,:,0], ssnew[2,:,0], 
#         ssnew[3,:,0]][...,0])
# print(hsamp[ssnew[0,:,1], ssnew[1,:,1], ssnew[2,:,1], 
#         ssnew[3,:,1]][...,1])
print(hsamp.shape)
print(ssnew.shape)
ampmax=hsamp[ssnew[0,], ssnew[1], ssnew[2], 
        ssnew[3]].squeeze()
ampmax2 = hsamp[tuple(ssnew)]
# why does this give me duplicates??
print(ampmax.shape)
print(ampmax)
print(max.shape)
print(max)

include r indices calculation

In [ ]:
# include also r indices: 
# Only way to avoid for loops over realizations?
#hsamp for all bins at all realizations

#gets max for each frequency and realization
max = np.amax(hsamp, axis=(0,1,2))
print(max)
shape = hsamp.shape
print('shape',shape)
newshape = (shape[0]*shape[1]*shape[2], shape[3], shape[4])
print('newshape', newshape)
hsamp = hsamp.reshape(newshape)
argmax = np.argmax(hsamp, axis=0)
hsamp = hsamp.reshape(shape)

# shape (3, F, R)
mqz = np.array(np.unravel_index(argmax, shape[:-2])) # unravel indices
# print('mqz', mqz.shape)

# MQZ in shape (3,F,R)
# Want to get MQZF in shape (4,F,R)
# Need f_ids in shape (1,F,R) to append
# Need r_ids in shape (1,F,R) to append
R=shape[4]
F=shape[3]
print('R =', R, 'F =',F)
f_ids = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int) # frequency indices
f_ids = np.repeat(f_ids, R).reshape(F,R)[np.newaxis,:]
r_ids = np.linspace(0,R-1,R).astype(int) # realization indices
print(r_ids)
r_ids = np.tile(r_ids, F).reshape(F,R)[np.newaxis,:]
print(r_ids, r_ids.shape)
ssnew = np.append(mqz,f_ids, axis=0)
ssnew = np.append(ssnew,r_ids, axis=0)
print(ssnew.shape)

In [ ]:
print(max.shape)

In [ ]:
print(hsamp[ssnew[0], ssnew[1], ssnew[2], ssnew[3], ssnew[4]])

In [ ]:
print(ssnew[0,0,0], ssnew[0,0,0].shape)
# mass index of all freqs, all reals
print(bgnum.shape)
print(bgnum[ssnew[0,0,0], ssnew[1,0,0], ssnew[2,0,0], ssnew[3,0,0],0])

bgnum

In [ ]:
r=1
print(bgnum[...,r], bgnum.shape)
print(hsamp.shape)
print(hsamp[ssnew[0], ssnew[1], ssnew[2], 
        ssnew[3],ssnew[4]][...,r])
bgnumcop = np.copy(bgnum)
# bgnumcop[ssnew[0,:,:], ssnew[1,:,:], ssnew[2,:,:], ssnew[3,:,:]]=9
bgnumcop[[ssnew[0], ssnew[1], ssnew[2], ssnew[3], ssnew[4]]]-=1
print(bgnumcop[...,r], bgnumcop.shape)

old arrs stuff

In [ ]:
arr = np.array([[[1,1], [2,2], [3,3]]])
print(arr, arr.shape)
app = np.array([[[0], [5], [6]]])
print(app, app.shape)
res = np.append(arr, app, axis=2)
print(res, res.shape)

# take shape (a,b,c) and append shape (a,b,d) to get shape (a,b,c+d)

In [ ]:
arr = np.array([[1],[2],[3]])
print(arr, arr.shape)
app = np.array([[0],[0],[3]])
print(app, app.shape)
res = np.append(arr, app, axis=1)
print(res, res.shape)

# take shape (a,b,c) and append shape (a,b,d) to get shape (a,b,c+d)
# I have shape (3,5,2) and I want to append ..... 
# to get shape (4,5,2)

In [ ]:
arr = np.array([[1,2],[3,4]])
print(arr, arr.shape)
app = np.array([[0,0]])
print(app, app.shape)
res = np.append(arr, app, axis=0)
print(res, res.shape)

# take shape (a,b,c) and append shape (a,b,d) to get shape (a,b,c+d)
# I have shape (3,5,2) and I want to append ..... 
# to get shape (4,5,2)

In [ ]:
arr = np.array([[1,2],[3,4]])
print(arr, arr.shape)
app = np.array([[0],[0]])
print(app, app.shape)
res = np.append(arr, app, axis=1)
print(res, res.shape)

# take shape (a,b,c) and append shape (a,b,d) to get shape (a,b,c+d)
# I have shape (3,5,2) and I want to append ..... 
# to get shape (4,5,2)

In [ ]:
arr = np.array([[[1],[2]],[[3],[4]],[[5],[6]]])
print(arr, arr.shape)
app = np.array([[[0],[0]]])
print(app, app.shape)
res = np.append(arr, app, axis=0)
print(res, res.shape)

# take shape (a,b,c) and append shape (a,b,d) to get shape (a,b,c+d)
# I have shape (3,5,2) and I want to append ..... 
# to get shape (4,5,2)

In [ ]:
print(mqz.shape) # mqz is shape (3,5,2) = (3,F,R)
# I want to append frequencies to get shape (4,5,2) = (4,F,R)
# this means I need my f_indices in shape (1,5,2)
print(f_ids.shape)
print(f_ids[np.newaxis,:,np.newaxis].shape)
print(np.tile(f_ids, (1,2)))
print(np.repeat(f_ids, 2).reshape(5,2).shape)

In [ ]:
R = 2
F = 5
reps = np.repeat(f_ids, R).reshape(F,R)
resh = reps[np.newaxis,:]
print(resh)
print(resh.shape)

f appending

In [ ]:
# I GOt IT!!!!!
# MQZ in shape (3,F,R)
# Want to get MQZF in shape (4,F,R)
# Need f_ids in shape (1,F,R) to append

f_ids = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int) # frequency indices
f_ids = np.repeat(f_ids, R).reshape(F,R)[np.newaxis,:]
mqzf = np.append(mqz,f_ids, axis=0)
print(mqzf.shape)

single realization

In [ ]:
hsamp = np.array([[[[1,0,0,0,0],[0,9,0,0,0],[0,0,0,0,0]]],

                 [[[0,0,0,0,0],[0,0,3,0,0],[0,0,0,4,5]]]])[...,np.newaxis]
print(hsamp.shape)
bgnum = np.full_like(hsamp, 100)


In [ ]:
# include also r indices: 
# Only way to avoid for loops over realizations?
#hsamp for all bins at all realizations

#gets max for each frequency and realization
max = np.amax(hsamp, axis=(0,1,2))
print(max)
shape = hsamp.shape
print('shape',shape)
newshape = (shape[0]*shape[1]*shape[2], shape[3], shape[4])
print('newshape', newshape)
hsamp = hsamp.reshape(newshape)
argmax = np.argmax(hsamp, axis=0)
hsamp = hsamp.reshape(shape)

# shape (3, F, R)
mqz = np.array(np.unravel_index(argmax, shape[:-2])) # unravel indices
# print('mqz', mqz.shape)

# MQZ in shape (3,F,R)
# Want to get MQZF in shape (4,F,R)
# Need f_ids in shape (1,F,R) to append
# Need r_ids in shape (1,F,R) to append
R=shape[4]
F=shape[3]
print('R =', R, 'F =',F)
f_ids = np.linspace(0,len(mqz[0])-1,len(mqz[0])).astype(int) # frequency indices
f_ids = np.repeat(f_ids, R).reshape(F,R)[np.newaxis,:]
print('f_ids',f_ids, f_ids.shape)
r_ids = np.linspace(0,R-1,R).astype(int) # realization indices
print('r_ids',r_ids,r_ids.shape)
r_ids = np.tile(r_ids, F).reshape(F,R)[np.newaxis,:]
print(r_ids, r_ids.shape)
ssnew = np.append(mqz,f_ids, axis=0)
ssnew = np.append(ssnew,r_ids, axis=0)
print(ssnew.shape)

In [ ]:
print(ssnew)

In [ ]:
r=0
print(bgnum[...,r], bgnum.shape)
print(hsamp.shape)
print(hsamp[ssnew[0], ssnew[1], ssnew[2], 
        ssnew[3],ssnew[4]][...,r])
bgnumcop = np.copy(bgnum)
# bgnumcop[ssnew[0,:,:], ssnew[1,:,:], ssnew[2,:,:], ssnew[3,:,:]]=9
bgnumcop[[ssnew[0], ssnew[1], ssnew[2], ssnew[3], ssnew[4]]]-=1
print(bgnumcop, bgnumcop.shape)

# From ss_real_dev 3/10

## Scratch

grid:

1x  |  xx  |  xx  |  xx  |  xx \
xx  |  9x  |  x3  |  xx  |  xx \
xx  |  xx  |  xx  |  x4  |  x5 

In [ ]:
grid = np.array([[[[1,0,0,0,0],[0,9,0,0,0],[0,0,0,0,0]]],
                 [[[0,0,0,0,0],[0,0,3,0,0],[0,0,0,4,5]]]])
print(grid.shape)
mqz_f, f_mqz, all_f, f_all= max_index_at_f(grid)
print('mqz_f:\n',mqz_f)
print('\nf_mqz:\n', f_mqz)
print('\nall_f:\n', all_f)
print('\nf_all:\n', f_all)

In [ ]:
maxes = np.amax(grid, axis=(0,1,2))
print(maxes)

hsmax = np.amax(grid, axis=(0,1,2)) #find max hs at each frequency
    
    #### 2) Record the indices and strain of that single source

# --- Indices of Loudest Bin ---
# Shape [F, 4], looks like
# [[m_idx,q_idx,z_idx,0],
#  [m_idx,q_idx,z_idx,1],
#   ........
#  [m_idx,q_idx,z_idx,F-2]]
ssidx = np.argwhere(grid==hsmax) 
for s in range(len(ssidx)):
    m,q,z,f = ssidx[s]
    print(m,q,z,f, 'grid max:',grid[m,q,z,f])
print(ssidx)

grid: argwhere vs where

In [ ]:
where = np.where(grid==hsmax)
where_arr = np.array([*where])
argwhere = np.argwhere(grid==hsmax)
print('where:\n', where)
print('np.where unpacked and put in numpy array\n', where_arr)
print('argwhere:\n', argwhere)
print('first point', argwhere[0,:], where_arr[:,0])
for f in range(len(argwhere)):
    print('f: by np.array[*where][:,f]', where_arr[:,f],  
          'or by argwhere[f,:]', argwhere[f,:])

grid: using reshape

In [ ]:
shape = grid.shape
print(shape)
print(grid)
grid = grid.reshape(6,5)
print(grid.shape)
print(grid)
grid = grid.reshape(shape)
print(grid.shape)
print(grid)

In [ ]:
shape = (2,1,3,5)
print(shape[:-1])
grid = grid.reshape(shape)
# print('grid:', grid.shape,'\n', grid)
grid = grid.reshape(6,5)
print('grid:', grid.shape,'\n', grid)
# print('grid[4,2]', grid[4,2])

argmax0 = np.argmax(grid,axis=0)
# argmax1 = np.argmax(grid,axis=1)
# unraveledargmax = np.unravel_index(argmax, grid.shape)
print('argmax0:', argmax0.shape, argmax0)
for i in range(len(grid[0])):
    print('argmax[i]=',argmax0[i], 
          'i=', i, 
          'grid[argmax[i], i]] =', grid[argmax0[i], i])
# print('argmax1:', argmax1.shape, argmax1)

# print('unraveled:', unraveledargmax)
# print('max at argmax:', grid[:,argmax])
# argidx = np.unravel_index(argmax, grid.shape)
print('grid[argmax,:]\n', grid[argmax0,:])
# print('grid[:,argmax]\n', grid[:,argmax0])
grid = grid.reshape(shape)
unraveled = np.unravel_index(argmax0, shape[:-1])
print('unraveled', len(unraveled), unraveled[0].shape, unraveled)
for i in range(len(grid[0,0,0,:])):
    m,q,z= unraveled[0][i], unraveled[1][i], unraveled[2][i] #, unraveled[3][i]
    print('unraveled[:,i] m,q,z,f = ', m,q,z,i)
    print(grid[m,q,z,i])
    # print('argmax[i]=',argmax0[i], 
    #       'i=', i, 
    #       'grid[argmax[i], i]] =', grid[n,q,z, i])

# print('grid:', grid.shape,'\n', grid)



figured out how to use my faster mqz indexing to get maxes from grid, with fancy indexing

In [ ]:
mqz = reshaped_argmax_at_f(grid)
print(mqz)
mqzf = mqzf_from_mqz(mqz, True)
print(mqzf)

In [ ]:
testarr = np.array([[0,0],[-1,-2],[3,4]])
print(testarr)
sorted = np.sort(testarr, axis=0)
print(sorted)

sort over all M,q,z for each individual F and R
np.sort( axis?)

* keep F and R indices separated

In [ ]:
faster = faster_mqzf_from_mqz(mqz, False)
print('faster:', faster)
print(grid.shape)
print('1st max at m,q,z,f = 0,0,0,0:', grid[0,0,0,0])
print('2nd max at m,q,z,f = 0,0,1,1:', grid[0,0,1,1])
print('3rd max at m,q,z,f = 1,0,1,2:', grid[1,0,1,2])
print('4th max at m,q,z,f = 1,0,2,3:', grid[1,0,2,3])
print('5th max at m,q,z,f = 1,0,2,4:', grid[1,0,2,4])
print(grid[[0,0,1,1,1], [0,0,0,0,0], [0,1,1,2,2], [0,1,2,3,4]])
fancy = [0,0,1,1,1], [0,0,0,0,0], [0,1,1,2,2], [0,1,2,3,4]
print('grid[fancy] =',grid[fancy])
print('fancy:', fancy)
print(grid[faster[0], faster[1], faster[2], faster[3]]) # this gives maxes!!! 

# listed = (faster[0], faster[1], faster[2], faster[3])
# print(listed)


using reshaped argmax, and mqzf to mqz methods

In [ ]:
# this is what mqz looks like
# all [[M indices], [q indices], [z indices]] in shape (3,F)
arr = np.array([[1,2,3,4,5], [10,20,30,40,50], [100,200,300,400,500]])
print(np.array(arr).shape)
# Here are the F indices
indices = np.array([0,1,2,3,4])

# I'd like to get this in the shape (4,F):
# [array of M indices, array of q indices, array of z indices]
newarr = np.append(arr,indices).reshape(4,len(indices))
print(newarr)

apply to example 3

In [ ]:
edges, number, fobs, exname = example2(print_test=False)
hc_bg, hc_ss, hsamp, ssidx, hsmax, bgnum, ssnew = ss_by_ndars(edges, number, realize=False, round=True)
# print(holo.utils.stats(hsamp))
np.set_printoptions(precision=2)
print('hsamp\n', hsamp)
print('hsmax', hsmax)
shape = hsamp.shape



In [ ]:
print('old hsamp', hsamp.shape, '\n', hsamp)

newshape = (shape[0]*shape[1]*shape[2], shape[3])
hsamp = hsamp.reshape(newshape)
print('new hsamp', hsamp.shape, '\n', hsamp)
argmax = np.argmax(hsamp, axis=0)
print('argmax', argmax)


hsamp = hsamp.reshape(shape)
print('m,q,z shape:', shape[:-1])
maxidx = np.unravel_index(argmax, shape[:-1])

for i in range(len(hsamp[0,0,0,:])):
    m,q,z= maxidx[0][i], maxidx[1][i], maxidx[2][i] #, unraveled[3][i]
    print('unraveled[:,i] m,q,z,f = ', m,q,z,i)
    print(hsamp[m,q,z,i], hsmax[i])


In [ ]:
# am = (np.argmax(hsamp[3,3,:,4]))
# print(hsamp[3,3,:,4])
# print(hsamp[3,3,am,4])

In [ ]:
# mqz_f, f_mqz, all_f, f_all= max_index_at_f(grid)
# argwhere = argwhere_at_f(grid)
# print(argwhere)
# assert (np.all(f_all == argwhere)), 'argwhere failing to find maxes'

In [ ]:
# for i in range(len(grid[0,0,0,:])):
#     m,q,z,f = argwhere[i]
#     print(m,q,z,f, 'max:', grid[m,q,z,f])

In [ ]:
# grid = np.array([[[[1,0,0,0,0],[0,2,0,0,0],[0,0,0,0,0]]],
#                  [[[0,0,0,0,0],[0,0,3,0,0],[0,0,0,4,5]]]])
# maxes = np.max(grid, axis=(0,1,2))
# # print('grid:', grid.shape, '\n', grid)
# # print('maxes', maxes.shape, '\n', maxes)
# where = np.where(grid==maxes)
# print('where: len=', len(where), 'with shapes=', where[0].shape, where[1].shape, where[2].shape,
#       where[3].shape, '\n', where)
# print('max by where:', grid[np.where(maxes==grid)])
# print()
# argmax = np.argmax(grid)
# m,q,z,f = np.unravel_index(np.argmax(grid), grid.shape)
# print('m:',m, '\nq:',q, '\nz:',z, '\nf:',f)
# print('argmaxes:', argmax.shape, '\n', argmax)

In [ ]:
# idxs = np.argwhere(arr==maxes)
# print('idxs', idxs.shape,'\n', idxs)
# rots = np.rot90(idxs)
# print('rots', rots.shape, '\n', rots)
# print(idxs[0])
# for i in [0,1]:
#     print(arr[idxs[i,0], idxs[i,1], idxs[i,2]])
# for i in [0,1]:
#     # print(arr[rots[0,i], rots[1,i], rots[2,i]])
#     print(arr[rots[2,i], rots[1,i], rots[0,i]])
# # the rot situation is trickier, just use
# # idxs = np.argwhere(arr==maxes)
# # returns (2,3) array
# # or (f, 4) array?


# Cython


In [ ]:
# %load ../init.ipy
%reload_ext autoreload
%autoreload 2
from importlib import reload

import holodeck as holo
import holodeck.single_sources as ss
import holodeck.cyutils as cy

reload(cy) # not working :/
reload(ss)

Using SS's

In [ ]:
edges, number, fobs, exname = ss.example2(print_test=False)
h2fdf = ss.h2fdf(edges)

In [ ]:
hc_ss, hc_bg, ssidx = cy.ss_bg_hc(number, h2fdf, 2)

## Compare to non cython results

Basic Tests Cython

In [ ]:
cy.test_long_array(5)

In [ ]:
arr1, arr2 = cy.test_multiple_arrays()

In [ ]:
arr = cy.test_2darray()

In [ ]:
arr = cy.test_1darray()
print(arr)

In [ ]:
import holodeck.single_sources as ss

In [ ]:
edges, number, fobs, exname = ss.example3(print_test=False)

### Check ssidx match

In [ ]:
hc_bg, hc_ss, hsamp, ssidx, hsmax, bgnum = ss.ss_by_ndars(edges, number, False, round=True)

In [ ]:
bg_c, ss_c, idx_c, amp_c = ss.ss_by_cdefs(edges, number, False, round=True)

In [ ]:
print(ssidx.squeeze())

In [ ]:
hc_bg_, hc_ss_l, hsamp_l, ssidx_l, hsmax_l, bgnum_l = ss.ss_by_loops(edges, number, False, round=True)

In [ ]:
print(ssidx_l.squeeze())

In [ ]:
print(hsamp[...,0],amp)

In [ ]:
print(holo.utils.stats(ssidx))
print(holo.utils.stats(idx))

In [ ]:
print(hc_ss.shape)

In [ ]:
marr, qarr, zarr, farr = ss.parameters_from_indices(edges, ssidx2)

In [ ]:
print(marr)

In [ ]:
print(ssidx)

In [ ]:
print(qarr)

In [ ]:
plt.plot(np.arange(len(marr)), marr)

In [ ]:
plt.plot(fobs, hc_ss[:,0])

In [ ]:
print(hc_ss.shape)

In [ ]:
hsamp = amp
print(hsamp)

In [ ]:
temp = np.copy(hsamp)


In [ ]:
hsamp=amp
print(hsamp.shape)
print(hsamp)

In [ ]:
shape = hsamp.shape
newshape = (shape[0]*shape[1]*shape[2], shape[3])
hsamp = hsamp.reshape(newshape) # change hsamp to shape (M*Q*Z, F)
argmax = np.argmax(hsamp, axis=0) # max at each frequency
hsamp = hsamp.reshape(shape) # restore hsamp shape to (M, Q, Z, F)
ssidx3 = np.array(np.unravel_index(argmax, shape[:-1])) # unravel indices


In [ ]:
print(ssidx3)

In [ ]:
print(hsamp[tuple(ssidx)])

# Timing. What data to store?

In [ ]:
%timeit edges, number, fobs, fname = ss.example5()

In [ ]:
import holodeck as holo
from holodeck.constants import YR, MSOL

In [ ]:
dur = 10.0*YR
cad = .2*YR
%timeit edges, number, fobs = ss.example(dur, cad, mtot=None, mrat=None, redz=None, print_test=False)

In [ ]:
%timeit ss.h2fdf(edges)

In [ ]:
h2fdf = ss.h2fdf(edges)
%timeit cy.ss_bg_hc(number, h2fdf, 30)

In [ ]:
hc_ss, hc_bg, ssidx, hsamp = ss.ss_by_cdefs(edges, number, 30)

In [ ]:
%timeit hc_ss, hc_bg, ssidx, hsamp = ss.ss_by_cdefs(edges, number, 30)

In [ ]:
%timeit marr, qarr, zarr, farr = ss.parameters_from_indices(edges, ssidx)
marr, qarr, zarr, farr = ss.parameters_from_indices(edges, ssidx)

function to calculate effective parameters

In [ ]:
print(hc_ss.shape)

In [ ]:
print(marr.shape)
print(np.average(marr, axis=1, weights=hc_ss))

In [ ]:
def effective_paramater(arr, hc_ss):
    """ 
    Calculate the average effective binary parameter, given the 
    """
    eff = np.average(arr, axis=1, weights=hc_ss**2)
    return eff

# Parameters in cython

In [ ]:
import kalepy as kale
import holodeck.cyutils as cy
import holodeck.single_sources as ss

edges, number, fobs, exname = ss.example5()
h2fdf = ss.h2fdf(edges)
mt = kale.utils.midpoints(edges[0]) #: total mass
mr = kale.utils.midpoints(edges[1]) #: mass ratio
rz = kale.utils.midpoints(edges[2]) #: redshift

In [ ]:
hc_ss, hc_bg, ssidx, bgpar, sspar = cy.ss_bg_hc_and_par(number, h2fdf, 30, mt, mr, rz)

In [ ]:
print(sspar[:,0,0])
print(bgpar[:,0,0])

In [ ]:
print(mt[ssidx[0,0,0]], mr[ssidx[1,0,0]], rz[ssidx[2,0,0]])

In [ ]:
%timeit hc_ss, hc_bg, ssidx, bgpar, sspar = cy.ss_bg_hc_and_par(number, h2fdf, 30, mt, mr, rz)

In [ ]:
%timeit hc_ss, hc_bg, ssidx, hsamp, bgpar, sspar = ss.ss_by_cdefs(edges, number, 30, params=True)

In [ ]:
%timeit hc_ss, hc_bg, ssidx, hsamp = ss.ss_by_cdefs(edges, number, 30)

In [ ]:
edges, number, fobs, exname = ss.example2()

In [ ]:
hc_bg, hc_ss, hsamp, ssidx, hsmax, bgnum = ss.ss_by_ndars(edges, number, realize=2)

In [ ]:
print(bgnum[:,:,:,2,0])
print(bgnum.shape)
together = np.array([hsamp, bgnum])
print(together.shape)
shape = together.shape
together = together.reshape((shape[0], shape[1]*shape[2]*shape[3], shape[4], shape[5]))
print(together.shape)
print(together[:,:,2,0])
sorted = np.sort(together, axis=1)
print('together\n', together[:,:,2,0])
print('hsort', hsort[:,2,0])
print('bgnum', bgnum.reshape((2*1*3, 5, 2))[:,2,0])
print('sorted hsort', np.sort(hsort, axis=0)[:,2,0])
print('sorted together\n', sorted[:,:,2,0])

In [ ]:
np.set_printoptions(precision=2)
print(hsamp.shape)
print(hsamp[:,:,:,2,0]) #0th realization, 2nd frequency
shape = hsamp.shape
hsort = hsamp.reshape(shape[0]*shape[1]*shape[2], shape[3], shape[4])
print(hsort[:,2,0])
print(np.sort(hsort, axis=0)[:,2,0])

In [ ]:
# shape (M, Q, F)
# sort over any M and Q for each individual F
# first frequency: m1,q1,f1 = 5; m1q2,f1 = 4, m2q1,f1 = 6, m2q2,f1 = 3
# second frequency: m1,q1,f2 = 20, m1q2f2 = 35, m2q1f2 = 10, m2q2f2 = 11
# order by first all frequencies of m1 q1, then all q1s for m1, 
test = np.array([[[5,20], [4,35]], [[6,10], [3,11]]])
goal = np.array([[3, 4, 5, 6], [10,11,20,35]])
print(test)
print(test.reshape((2*2, 2)))
print(np.sort(test.reshape(2*2, 2), axis=0))
# print(np.sort(test, axis=0))
# print(goal)

In [ ]:
print(grid)

In [ ]:
print(hsamp[])

## Cython sorting

In [ ]:
import holodeck.cyutils as cc
import numpy as np
import holodeck as holo
import holodeck.single_sources as ss

meshgrid, defining random h2fdf and bgnum

In [ ]:
M = 2
Q = 1
Z = 3
F = 2
h2fdf = np.zeros((M,Q,Z,F))
bgnum = np.zeros((M,Q,Z,F))
mqz = np.zeros((3,M,Q,Z))
for mm in range(M):
    for qq in range(Q):
        for zz in range(Z):
            mqz[0,mm,qq,zz] = mm
            mqz[1,mm,qq,zz] = qq
            mqz[2,mm,qq,zz] = zz
            for ff in range(F):
                h2fdf[mm,qq,zz,ff] = np.random.uniform(1,10)
                bgnum[mm,qq,zz,ff] = int(np.random.uniform(0,5))
                
print(h2fdf)
print(bgnum)
m_i = np.arange(M) 
q_i = np.arange(Q)
z_i = np.arange(Z)
m_idx, q_idx, z_idx = np.meshgrid(m_i, q_i, z_i, indexing='ij')

In [ ]:
print(mqz[0,:,:,:] == m_idx)
print(mqz[1,:,:] == q_idx)
print(mqz[2,:,:]==z_idx)
print(np.all(mqz==np.array([m_idx, q_idx, z_idx])))
# meshgrid for indeing works!

In [ ]:
cc.sort_h2fdf(h2fdf)

In [ ]:
copy = np.copy(h2fdf.reshape(M,Q,Z,F))
h2fdf = h2fdf.reshape(M*Q*Z, F)

bgnum = bgnum.reshape(M*Q*Z, F)
m_idx = m_idx.reshape(M*Q*Z)
q_idx = q_idx.reshape(M*Q*Z)
z_idx = z_idx.reshape(M*Q*Z)

indices = np.argsort(-h2fdf, axis=0)
print(indices.shape)
print(h2fdf.shape)
for ff in range(len(h2fdf[0])):
    print('\nff =', ff)
    for hh in range(len(h2fdf)):
        ii = indices[hh,ff]
        mm = m_idx[ii]
        qq = q_idx[ii]
        zz = z_idx[ii]
        print(h2fdf[ii,ff],copy[mm,qq,zz,ff])
        print(mm,qq,zz, np.unravel_index(ii, (M,Q,Z)))
        

In [ ]:
unraveled = np.array(np.unravel_index(indices, (M,Q,Z)))
print((unraveled).shape)
msort = unraveled[0,:,:]
qsort = unraveled[1,:,:]
zsort = unraveled[2,:,:]
print(msort.shape)

In [ ]:
print(h2fdf.shape)

In [ ]:
for ff in range(len(h2fdf[0])):
    print('\nff =', ff)
    for hh in range(len(h2fdf)):
        ii = indices[hh,ff]
        mm = m_idx[ii]
        qq = q_idx[ii]
        zz = z_idx[ii]
        print(h2fdf[ii,ff],copy[mm,qq,zz,ff])
        print(mm,qq,zz, np.unravel_index(ii, (M,Q,Z)), msort[hh,ff], qsort[hh,ff], zsort[hh,ff])

pass cython 
h2fdf [MxQxZ, F]
sorted [MxQxZ, F] (argsort)
edges [M,Q,Z,F]
sorted []

In [ ]:
# summary so far:
# setup random example h2fdf
M = 2
Q = 1
Z = 3
F = 2
h2fdf = np.zeros((M,Q,Z,F))
bgnum = np.zeros((M,Q,Z,F))
for mm in range(M):
    for qq in range(Q):
        for zz in range(Z):
            for ff in range(F):
                h2fdf[mm,qq,zz,ff] = np.random.uniform(1,10)
                bgnum[mm,qq,zz,ff] = int(np.random.uniform(0,5))

In [ ]:
# sort and index
indices = np.argsort(-h2fdf.reshape(M*Q*Z, F), axis=0)
unraveled = np.array(np.unravel_index(indices, (M,Q,Z)))
msort = unraveled[0,:,:]
qsort = unraveled[1,:,:]
zsort = unraveled[2,:,:]


In [ ]:
# test
print(h2fdf.shape)
for ff in range(F):
    for hh in range(len(indices)-1):
        # cur = h2fdf[indices[hh], ff]
        # print(h2fdf[msort[hh,ff], qsort[hh,ff], zsort[hh,ff], ff])
        # true of any sort
        assert((h2fdf[msort[hh,ff], qsort[hh,ff], zsort[hh,ff], ff]) > (h2fdf[msort[hh+1,ff], qsort[hh+1,ff], zsort[hh+1,ff], ff]))
        
        # true for a real h2fdf (same across frequencies)
        assert(msort[hh,0]==msort[hh,1])
        assert(qsort[hh,0]==qsort[hh,1])
        assert(zsort[hh,0]==zsort[hh,1])


try for example 5

In [ ]:
edges, number, fobs, exname = ss.example5()
h2fdf = ss.h2fdf(edges)

In [ ]:
M = number.shape[0]
Q = number.shape[1]
Z = number.shape[2]
F = number.shape[3]
print(M,Q,Z,F)

In [ ]:
# sort and index
indices = np.argsort(-h2fdf.reshape(M*Q*Z, F), axis=0)
unraveled = np.array(np.unravel_index(indices, (M,Q,Z)))
msort = unraveled[0,:,:]
qsort = unraveled[1,:,:]
zsort = unraveled[2,:,:]


In [ ]:
# test
print(h2fdf.shape)
for ff in range(F):
    for hh in range(len(indices)-1):
        # cur = h2fdf[indices[hh], ff]
        # print(h2fdf[msort[hh,ff], qsort[hh,ff], zsort[hh,ff], ff])
        # true of any sort
        assert((h2fdf[msort[hh,ff], qsort[hh,ff], zsort[hh,ff], ff]) > (h2fdf[msort[hh+1,ff], qsort[hh+1,ff], zsort[hh+1,ff], ff]))
        
        # true for a real h2fdf (same across frequencies)
        assert(msort[hh,0]==msort[hh,1])
        assert(qsort[hh,0]==qsort[hh,1])
        assert(zsort[hh,0]==zsort[hh,1])


so actually, we don't need to do it for every frequency

In [ ]:
# sort and index
indices = np.argsort(-h2fdf[...,0].flatten()) # just sort for first frequency
unraveled = np.array(np.unravel_index(indices, (M,Q,Z)))
print(unraveled.shape)
msort = unraveled[0,:]
qsort = unraveled[1,:]
zsort = unraveled[2,:]
print(msort.shape)

In [ ]:
# test
print(h2fdf.shape)
for ff in range(F):
    for hh in range(len(indices)-1):
        # cur = h2fdf[indices[hh], ff]
        # print(h2fdf[msort[hh,ff], qsort[hh,ff], zsort[hh,ff], ff])
        # true of any sort
        assert((h2fdf[msort[hh,ff], qsort[hh,ff], zsort[hh,ff], ff]) > (h2fdf[msort[hh+1,ff], qsort[hh+1,ff], zsort[hh+1,ff], ff]))
        
        # true for a real h2fdf (same across frequencies)
        assert(msort[hh,0]==msort[hh,1])
        assert(qsort[hh,0]==qsort[hh,1])
        assert(zsort[hh,0]==zsort[hh,1])
